In [1]:
# XML, TEI, CMI/F and data handling
from bs4 import BeautifulSoup # Hent BeautifulSoup-modulen (https://www.crummy.com/software/BeautifulSoup/) for XML
from bs4 import Comment # BS4-addon for å håndtere kommentarer <!-- X -->
import re # Regex
import pandas as pd
import collections # Facilitate dynamic dict

# Time and date
import datetime # Dates
from datetime import date
import time # Time

# File and folder handling
import glob # The yeast of thought and mind
import os # Filsystem; mapper, lagring, åpning, etc...
import shutil # Se os+

from string import punctuation


#Check if *keys (nested) exists in dict
def keys_exists(element, *keys):
    if not isinstance(element, dict):
        raise AttributeError('keys_exists() expects dict as first argument.')
    if len(keys) == 0:
        raise AttributeError('keys_exists() expects at least two arguments, one given.')

    _element = element
    for key in keys:
        try:
            _element = _element[key]
        except KeyError:
            return False
    return True

today = date.today() # Sett dato i dag
today = today.strftime("%Y-%m-%d") # Formater dato
currVer = "MXMLM Preprocessor"
print("Version",currVer)
cmifUid = "a403c593-09df-4538-8acf" # Unique ID. Used in sourceDesc of CMIF. Don't change it without a good reason.
# cmifUid is also used as "source" for the time being in each object. Read more about this in CMIF docs.

# *MunchXMLmuncher* **X2**
# MXMLMx2 reads and preprocesses files. First off, create subdirectories and locate our files:

Version MXMLM Preprocessor


In [2]:
hasChrono,hasXMLs = False,False

In [3]:
# Chrono
# Next, let's get the chrono if it exists, and then read the data out of it to a dict.

In [4]:
listofbaddies = []
lookupChrono = sorted(glob.glob("*Kronologi_Munchs_brev*.xlsx"), key=os.path.getmtime)
x = len(lookupChrono)-1
if x > -1:
    print("Newest chronology file:",lookupChrono[x])
    chronologyFile = lookupChrono[x]
    hasChrono = True
    #shutil.copy2(chronologyFile, inputfolder+"/kronologi.xlsx")
    #chronologyFile = inputfolder+"/kronologi.xlsx"
    
    chronology = pd.read_excel(chronologyFile).dropna(axis=1, how='all').dropna(axis=0, how='all').reset_index(drop=True)
    chronology = chronology.fillna("N/A")
    
    try:
        if CHRONODICT:
            print("CHRONODICT found with content")
        else:
            print("CHRONODICT found without content")
    except:
        CHRONODICT = collections.defaultdict(dict)
        print("CHRONODICT created")
    for idx,row in chronology.iterrows():
        formattingError = False
        document = chronology.iloc[idx]['Objektnr.']
        rawdate = chronology.iloc[idx]['Dato']
        if rawdate != "N/A" and document != "N/A": # If date and documents are not N/A
            chkStr = document[0:2]
            if chkStr == "PN":
                #print(document)
                document = document.replace(" ","0")
                if len(document) == 5:
                    filenameEdited=document[:2] + "0" + document[len(document)-3:]
                else:
                    filenameEdited = document[:2] +"000"+ document[len(document)-4:]
                    filenameEdited = filenameEdited[:2] + filenameEdited[len(filenameEdited)-4:]
                filenamePlain = filenameEdited[0:6]
            elif chkStr == "MM":
                if "," in document or "+" in document: # If the document title has a comment in it with either of these...
                    #print("> Breaking",document)
                    if "," in document:
                        breakup = document.split(",")
                    elif "+" in document:
                        breakup  = document.split("+")
                    document = breakup[0] # Break the title and try to use the first part onwards.
                
                #No-MM_K0499
                #filenamePlain = document[0:11]
                if len(document) == 8:
                    #print(document)
                    filenamePlain = document[:2]+"_"+document[3]+"0"+document[5:]
                elif len(document) == 10:
                    filenamePlain = document[:2]+"_"+document[3]+document[6:]
                else:
                    filenamePlain = document[:2]+"_"+document[3]+document[5:]
                filenamePlain = "No-"+filenamePlain
                #print(document,len(document),"/",filenamePlain,len(filenamePlain))
            elif chkStr == "M ": # Implemented to catch "M N 2714"...
                filenamePlain = document[:1]+"_"+document[2]+document[4:]
                filenamePlain = "No-M"+filenamePlain
                #filenamePlain = filenamePlain.replace(" ","")
                #print("Caught",document,">",filenamePlain)
            elif chkStr == "N ": # Implemented to catch "N 3491"...
                  # This is ASSUMED to be "No-MM_N3491" and not PN3491
                filenamePlain = "No-MM_N"+document[2:]
                #print("Caught",document,">",filenamePlain)
            elif chkStr == " M": # Implemented to catch " MM N 723"...
                filenamePlain = "No-"+document[1:3]+"_"+document[4]+"0"+document[6:]
                #print("Caught",document,">",filenamePlain)
            else:
                print("!!\tError in",document,"length:",len(document))
                #filenamePlain = "Formatting error"+document
                formattingError = True # Formatting error due to invalid filename.
            #print(filenamePlain)
            if len(filenamePlain) != len("No-MM_N0000"): # If string doesn't match with No-MM_N0000
                if len(filenamePlain) != len("PN0000"): # If string doesn't match with PN0000
                    #filenamePlain = "Formatting error"+filenamePlain # it is invalid.
                    formattingError = True # Formatting error due to invalid filename.
                    listofbaddies.append(filenamePlain+" ("+document+")")
            document = filenamePlain
            if isinstance(rawdate,datetime.date): # If it's just a datetime object
                #dateobject = rawdate.strftime("%Y-%m-%d")
                newdate = rawdate.strftime("%Y-%m-%d")
            else:
                #print(document,rawdate)
                dateobject = str(rawdate) # Make sure it's string
                
                string4print = document+" "+dateobject
                
                dateobject = dateobject.replace("–","-") # Replace long dash – with a normal dash -
                if ".-" in dateobject: # Like: 04.-05.1922.
                    dateobject = re.sub(r'^.*?\.-', '', dateobject, flags=re.DOTALL) # Remove EVERYTHING to the left of it.
# If you want date ranges, use complete dates separated by "-".
# Using ".-" to denote several things is not good.
                if "?" in dateobject:
                    dateobject = dateobject.replace("?","").strip(punctuation) # Remove n ?s and then also remove excess .

                    dateobject = dateobject.replace("..",".")
                if "-" in dateobject:
                    splitToFrom = dateobject.split("-")
                    fromDate = splitToFrom[0]
                    toDate = splitToFrom[1]
                    newFromDate = fromDate.split(".")
                    if isinstance(newFromDate,list):
                        newdate = newFromDate[len(newFromDate)-1]
                        for x in reversed(newFromDate):
                            if len(x) == 4:
                                pass
                            else:
                                if len(x) == 2:
                                    newdate+="-"+x
                                else:
                                    break  
                    else:
                        newdate = fromDate
                    newdate+="%"
                    newToDate = toDate.split(".")
                    if isinstance(newToDate,list):
                        newdateA = newToDate[len(newToDate)-1]
                        for x in reversed(newToDate):
                            if len(x) == 4:
                                pass
                            else:
                                if len(x) == 2:
                                    newdateA+="-"+x
                                else:
                                    break  
                    else:
                        newdateA = toDate
                    newdate+=newdateA
                    #print("yolo",document)
                else:
                    datelements = dateobject.split(".")
                    newdate = datelements[len(datelements)-1]
                    for x in reversed(datelements):
                        if len(x) == 4:
                            pass
                        else:
                            if len(x) == 2:
                                newdate+="-"+x
                            else:
                                break      
                #print(document,rawdate,"("+newdate+")")
            if formattingError is True:
                print(document,"is not a valid document ID and was excluded.")
            else:
                CHRONODICT[document]["date"] = newdate # Set the dict item's date to newdate

Newest chronology file: Kronologi_Munchs_brev_20220923.xlsx
CHRONODICT created
No-MM_K? is not a valid document ID and was excluded.
No-MM_K99 is not a valid document ID and was excluded.
No-MM_K27 is not a valid document ID and was excluded.
No-MM_N2 is not a valid document ID and was excluded.


In [5]:
## Places
# With the dates safely stored away, let's go for some place names.
if os.path.isfile("ID_sted-verdier.xlsx"):
    print("Detected ID_sted-verdier.xlsx")
    if os.path.exists("xml-filer"):
        print("XML>CMIF placename augmentation enabled")
        listXMLfiles = glob.glob("xml-filer/**/*.xml",recursive=True)
        hasXMLs = True
    else:
        print("No XML files provided. XML>CMIF placename augmentation disabled")
else:
    print("No ID_sted-verdier file provided. Skipping placename augmentation.")

Detected ID_sted-verdier.xlsx
XML>CMIF placename augmentation enabled


In [6]:
#n = 0
if hasXMLs == True: # If the XML files should be used for updating
    # Check to see if CHRONODICT is alive or not. If it is, use it as destination. If it isn't, create it.
    try:
        if CHRONODICT:
            print("CHRONODICT found with content")
        else:
            print("CHRONODICT found without content")
    except:
        CHRONODICT = collections.defaultdict(dict)
        print("CHRONODICT created")

    addrsFoundInXMLs = [] # Make a simple list to hold the short names of every file we've found addresses for
    xmlswithnoaddress = [] # Simple list for XMLs that have no address that can be printed later.
    plainnames = [] # Simple list for plain names of XMLs that have been checked.
    i=0
    for item in listXMLfiles:
        addrKey = "NONE" # Just in case
        find_address = [] # Ensure that this list clears on start of each item
        findFileName = item.split("\\") # Make filepath a list
        findFileName = findFileName[len(findFileName)-1] # Get the path destination file
        
        chkStr = findFileName[0:2] # Check the incoming ID - it's either PN+4 positions, or No+9 positions long.
        if chkStr == "PN": # 6 positions
            filenamePlain = findFileName[0:6] # Must be bounded to remove .xml as well as pagination from filename
            #print(findFileName,filenamePlain)
        elif chkStr == "No": # 11 positions
            filenamePlain = findFileName[0:11] # Must be bounded to remove .xml as well as pagination from filename
            #print(findFileName,filenamePlain)
        else: # This doesn't occur unless you've got files that don't belong here
            print("PROBLEM IN NAME PROCESSING",findFileName,chkStr) 
        if filenamePlain not in plainnames: # For every unique name, add to plainnames
            plainnames.append(str(filenamePlain)) # Just in case we need them later
        print ("\r","Progress:",round(i/len(listXMLfiles)*100),"%", end='\t')
        #print ("\r",n*0.1, end='')
        #n+=1
        if filenamePlain in addrsFoundInXMLs: # if we found the address for this xml
            pass # Skip if we've already found an address for this XML filename
        else:
            with open(item, "r", encoding="utf-8") as file: # Open a file
                letterfile = file.readlines() # Les innholdet som linjer
                letterfile = "".join(letterfile) # Linjene blir kombinert i en variabel
            soup = BeautifulSoup(letterfile, features="xml") # It is now soup
            #find_address = soup.findAll("address") # Look for addrline element.
            #augh,addrKey = "","" # Reset
            foundSender,foundRecipient = False,False # Reset

            ## Code below enables retrieval of an address enclosed in a dateline element.
            ## This is understood to be the sender's address.
            if foundSender == False: # If a recipient has not been found
                find_address = soup.find("dateline") # Look for a dateline element
                if find_address: # If there is a dateline element:
                    #print("Dateline in",findFileName)
                    find_address = find_address.findChild("placeName", recursive=True) # Get the placename
                    #print("DATELINE",find_address)
                    try: # There are documents with datelines but no locations in them confirmed.
                        addrKey = find_address.get('key') # Get the internal ID of the placename
                        try:
                            addrKey = addrKey.replace("pl","") # Remove "pl" prefix
                            addrsFoundInXMLs.append(filenamePlain) # Add the filename to the list of XMLs already found
                            #placenamedict[filenamePlain]["location"] = addrKey
                            CHRONODICT[filenamePlain]["location"] = addrKey
                            foundSender = True
                        except:
                            print("Problem with appending")

                    except:
                        if filenamePlain not in xmlswithnoaddress:
                            xmlswithnoaddress.append(filenamePlain)
                        #print("Dateline without address in",findFileName)
                else:
                    if filenamePlain not in xmlswithnoaddress:
                        xmlswithnoaddress.append(filenamePlain)
        i+=1
print("\t")

CHRONODICT found with content
 Progress: 100 %		1 %	 %	 %	 4 %	 5 %	%	 5 %	 7 %		8 %	%	 11 %	 12 %	 %	 15 %	%	 %	 16 %	 18 %		%		22 %	%	 %	 25 %	26 %		 38 %		 41 %	 41 %	 42 %	 43 %	 %	43 %		%	%	50 %	 52 %		 54 %		 %	 %	 %	 67 %			 %	 %	 74 %		%		 81 %		 87 %	 87 %	89 %	 %	 96 %	 %	%	


In [7]:
if hasXMLs == True:
    placenamedf = pd.read_excel("ID_sted-verdier.xlsx").dropna(axis=1, how='all').dropna(axis=0, how='all').reset_index(drop=True)
    placenamedf = placenamedf.fillna("N/A")
    #for item in placenamedict:
    for item in CHRONODICT:
        if keys_exists(CHRONODICT,item,'location'):
            #print("\tSent from:",CHRONODICT[item]['location'])
            #print(item)
            try:
                placenameSearch = placenamedf[placenamedf['ID'].astype(str) == str(CHRONODICT[item]['location'])]
                stedsnavn = placenameSearch["sted"].values[0]
                regionnavn = placenameSearch["region, nasjonal"].values[0]
                landnavn = placenameSearch["land"].values[0]
                kontinent = placenameSearch["region, internasjonal"].values[0]
                try:
                    if stedsnavn != "N/A":
                        #print("\tSted:",stedsnavn)
                        CHRONODICT[item]['location'] = stedsnavn
                    else:
                        if regionnavn != "N/A":
                            #print("\tRegion:",regionnavn)
                            CHRONODICT[item]['location'] = regionnavn
                        else:
                            if landnavn != "N/A":
                                #print("\tLand:",landnavn)
                                CHRONODICT[item]['location'] = landnavn
                            else:
                                if kontinent != "N/A":
                                    #print("\tKontinent:",kontinent)
                                    CHRONODICT[item]['location'] = kontinent
                                else:
                                    print("\tFant ikke stedsnavn.")
                except:
                    print("\t\tCouldn't match, I guess?")
            except:
                print("\t\tNo match for",str(CHRONODICT[item]['location']))
        #print("\n")

In [10]:
df = pd.DataFrame.from_dict(CHRONODICT)
chronoDF = df.T.fillna("N/A").reset_index(drop=False)
# Check for presence of date column (otherwise, would crash if no chronology used)
if 'date' not in chronoDF.columns:
    chronoDF['date'] = np.nan
    print("Filled date column with NaN")
# Check for presence of location column (otherwise, would crash if no letter XMLs used)
if 'location' not in chronoDF.columns:
    chronoDF['location'] = np.nan
    print("Filled location column with NaN")
chronoDF = chronoDF.rename(columns={"index": "document"}).sort_values(by=['document']).reset_index(drop=True)
places = 0
dates = 0
items = 0
for idx,row in chronoDF.iterrows():
    if row['date'] == "N/A" and row['location'] == "N/A":
        print(idx)
    else:
        #print(idx)
        items += 1
        if row['date'] != "N/A":
            #print(row['date'])
            dates += 1
        if row['location'] != "N/A":
            #print(row['location'])
            places += 1
        #print("-")
print(places,"places and",dates,"dates added over a total of",items,"items.")
chronoDF.to_csv("preprocessed.csv", sep=',', encoding='utf-8',index=False)
print("Preprocessing complete. Saved to preprocessed.csv. Printing report as Preprocessor Report.txt.")

if i > 0:
    pass
else:
    i=0

goodstring = "MXMLM Preprocessor run on date "+str(today)+"."
try:
    if plainnames:
        goodstring += "\nChecked "+str(i)+" files, of which "+str(len(plainnames))+" were identified as letters, of which "+str(len(addrsFoundInXMLs))+" had addresses."
    else:
        goodstring += "\nCAUTION Checked "+str(i)+" (zero?) files, of which "+str(len(plainnames))+" were identified as letters, of which "+str(len(addrsFoundInXMLs))+" had addresses."
except:
    goodstring+= "\nDidn't use placename augmentation?"
goodstring+="\n"+str(places)+" places and "+str(dates)+" dates added over a total of "+str(items)+" items."
if len(listofbaddies) > 0:
    errorstring = "Bad document IDs from the Chronology that could not be resolved:\n"
    for x in listofbaddies:
        errorstring+="\""+x+"\" "
    errorstring = errorstring.rstrip()
else:
    errorstring = "Didn't identify any bad document IDs that couldn't be resolved."
outputstring = goodstring+"\n"+errorstring

with open("Preprocessor Report.txt", "w", encoding="utf-8") as output_file:
    output_file.write(outputstring)